In [ ]:
import os
import json
import time
import random
import pandas as pd
from typing import Any, Dict, List, Optional

from openai import OpenAI

# -----------------------
# 0) Config
# -----------------------
INPUT_XLSX = "search_queries.xlsx"  # or "/mnt/data/search_queries.xlsx"
OUTPUT_JSONL = "sample100_outputs.jsonl"
OUTPUT_CSV = "sample100_flattened.csv"

N_SAMPLE = 100
RANDOM_SEED = 42

MODEL = "gpt-4o-mini"  # pick a model your org allows; keep consistent for evals

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# -----------------------
# 1) Your prompt template
# -----------------------
SYSTEM_PROMPT = """You are an Expedia Paid Marketing & Brand Query Understanding Engine.

Goal: Extract structured intent signals from a raw travel search query to route users to the most helpful landing page.

Rules:
- Do NOT hallucinate. Prefer null over weak inference.
- Infer only when obvious from keywords.
- Use bounded categories only.
- Separate extraction from scoring.
- If uncertain: value = null, confidence = low.

INPUT:
{user_query}

TASK:

1) Preprocessing
- Remove English stopwords
- token_count = remaining tokens
- is_more_than_80_percent_english = true if ≥80% tokens are English

2) Layer 1: Routing & Inventory (extract value | importance | confidence)
- intent_vertical_primary ∈ {{Stays, Commute, Activities, Packages, Vehicle Rental, Misc}} (10)
  (Infer synonyms: rooms/ house/airbnb/home stays/hotel/ b & b/ b and b/ break and breakfast/villa/condo, casa/hacienda, or anything synonymous to stay or hotel related→Stays;
Anything synonymous to mode of travel like bus or flight or train booking→Commute,
experiences/adventures/tour→Activities; car rental/ pickup vehicle→Vehicle Rental;
destination-only or vague→Misc, bundles/multi category → package )
- destination_info (geo/location/address or null) (10)
- trip_start_date (ISO or null)
- trip_end_date (ISO or null) (9)
  (Infer from terms like today, tomorrow, next week/month; single-day → same start/end)
- other_verticals (list or empty; additional verticals after first mention) (4)

3) Layer 2: Disambiguation & Page Template
- product_type ∈ {{stays, hotel, resort, rooms, airbnb, rentals, flight, train, bus, uber, lift, vehicle rental, tour, adventure, attraction, experience, package, misc}} (8)
- price_bucket ∈ {{budget, mid, luxury, unknown, deals, refundable, cancellable}} (8)
- booking_stage ∈ {{ideas, inspiration, exploratory, planning, booking, urgent}} (7)
- trip_theme ∈ {{family, business, group, romantic, adventure, relaxation}} (6)

4) Layer 3: Filters & UX Signals
- is_multi_traveler ∈ {{true, false, null}} (6)
- has_specific_requirements ∈ {{true, false}} (5)
  (e.g., pool, wifi, guide, ratings)

5) Query Richness Score
- Initialize score = 0
- For each extracted feature with confidence ≠ low:
    add normalized importance × confidence
- token_bonus:
    ≤2:+0.05 | 3–5:+0.10 | 6–8:+0.15 | >8:+0.20
- Normalize & clip final score to [0,1]

6) Output
Return ONLY this JSON:

{
  "token_count": int,
  "is_more_than_80_percent_english": boolean,
  "layer_1": {
    "intent_vertical_primary": {"value":..., "importance":10, "confidence":...},
    "destination_info": {"value":..., "importance":10, "confidence":...},
    "trip_start_date": {"value":..., "importance":9, "confidence":...},
    "trip_end_date": {"value":..., "importance":9, "confidence":...},
    "other_verticals": {"value":[...], "importance":4, "confidence":...}
  },
  "layer_2": {
    "product_type": {"value":..., "importance":8, "confidence":...},
    "price_bucket": {"value":..., "importance":7, "confidence":...},
    "booking_stage": {"value":..., "importance":7, "confidence":...},
    "trip_theme": {"value":..., "importance":6, "confidence":...}
  },
  "layer_3": {
    "has_specific_requirements": {"value":true/false, "importance":5, "confidence":...},
    "is_multi_traveler": {"value":true/false/null, "importance":6, "confidence":...}
  },
  "query_richness_score": float
}

Constraints:
- Output JSON only
- No extra fields
- Null > wrong inference
"""

# -----------------------
# 2) JSON Schema (Structured Outputs)
# -----------------------
# OpenAI Structured Outputs supports forcing outputs to match a JSON schema. :contentReference[oaicite:1]{index=1}
SCHEMA: Dict[str, Any] = {
  "name": "expedia_query_intent_schema",
  "schema": {
    "type": "object",
    "additionalProperties": False,
    "required": [
      "token_count",
      "is_more_than_80_percent_english",
      "layer_1",
      "layer_2",
      "layer_3",
      "query_richness_score"
    ],
    "properties": {
      "token_count": {"type": "integer"},
      "is_more_than_80_percent_english": {"type": "boolean"},
      "layer_1": {
        "type": "object",
        "additionalProperties": False,
        "required": ["intent_vertical_primary", "destination_info", "trip_start_date", "trip_end_date", "other_verticals"],
        "properties": {
          "intent_vertical_primary": {"$ref": "#/$defs/field10_vertical"},
          "destination_info": {"$ref": "#/$defs/field10_any"},
          "trip_start_date": {"$ref": "#/$defs/field9_date"},
          "trip_end_date": {"$ref": "#/$defs/field9_date"},
          "other_verticals": {"$ref": "#/$defs/field4_list"}
        }
      },
      "layer_2": {
        "type": "object",
        "additionalProperties": False,
        "required": ["product_type", "price_bucket", "booking_stage", "trip_theme"],
        "properties": {
          "product_type": {"$ref": "#/$defs/field8_product"},
          "price_bucket": {"$ref": "#/$defs/field7_price"},
          "booking_stage": {"$ref": "#/$defs/field7_stage"},
          "trip_theme": {"$ref": "#/$defs/field6_theme"}
        }
      },
      "layer_3": {
        "type": "object",
        "additionalProperties": False,
        "required": ["has_specific_requirements", "is_multi_traveler"],
        "properties": {
          "has_specific_requirements": {"$ref": "#/$defs/field5_bool"},
          "is_multi_traveler": {"$ref": "#/$defs/field6_multi"}
        }
      },
      "query_richness_score": {"type": "number"}
    },
    "$defs": {
      "confidence": {"type": "string", "enum": ["low", "medium", "high"]},

      "field10_vertical": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": ["string", "null"], "enum": ["Stays","Commute","Activities","Packages","Vehicle Rental","Misc", None]},
          "importance": {"type": "integer", "enum": [10]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field10_any": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": ["string", "null"]},
          "importance": {"type": "integer", "enum": [10]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field9_date": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": ["string", "null"]},  # ISO date string if present
          "importance": {"type": "integer", "enum": [9]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field4_list": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": "array", "items": {"type": "string"}},
          "importance": {"type": "integer", "enum": [4]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field8_product": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {
            "type": ["string", "null"],
            "enum": ["stays","hotel","resort","rooms","airbnb","rentals","flight","train","bus","uber","lift","vehicle rental",
                     "tour","adventure","attraction","experience","package","misc", None]
          },
          "importance": {"type": "integer", "enum": [8]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field7_price": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": ["string", "null"], "enum": ["budget","mid","luxury","unknown","deals","refundable","cancellable", None]},
          "importance": {"type": "integer", "enum": [7]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field7_stage": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": ["string", "null"], "enum": ["ideas","inspiration","exploratory","planning","booking","urgent", None]},
          "importance": {"type": "integer", "enum": [7]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field6_theme": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": ["string", "null"], "enum": ["family","business","group","romantic","adventure","relaxation", None]},
          "importance": {"type": "integer", "enum": [6]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field5_bool": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": "boolean"},
          "importance": {"type": "integer", "enum": [5]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      },

      "field6_multi": {
        "type": "object",
        "additionalProperties": False,
        "required": ["value", "importance", "confidence"],
        "properties": {
          "value": {"type": ["boolean", "null"]},
          "importance": {"type": "integer", "enum": [6]},
          "confidence": {"$ref": "#/$defs/confidence"}
        }
      }
    }
  }
}

# -----------------------
# 3) Call model (LLM-as-judge)
# -----------------------
def call_llm(query: str, max_retries: int = 5) -> Dict[str, Any]:
    prompt = SYSTEM_PROMPT.format(user_query=query)

    # Responses API Structured Outputs shape uses text.format with json_schema. :contentReference[oaicite:2]{index=2}
    for attempt in range(max_retries):
        try:
            resp = client.responses.create(
                model=MODEL,
                input=[{"role": "system", "content": prompt}],
                text={
                    "format": {
                        "type": "json_schema",
                        "json_schema": SCHEMA,
                        "strict": True
                    }
                },
                temperature=0
            )

            # The SDK returns parsed JSON in output_text for json_schema.
            # Depending on SDK version, you may access:
            # - resp.output_text (string JSON)
            # - resp.output[0].content[0].text (string JSON)
            raw = getattr(resp, "output_text", None)
            if raw is None:
                raw = resp.output[0].content[0].text

            return json.loads(raw)

        except Exception as e:
            # exponential backoff for rate limits / transient errors
            sleep_s = min(10, (2 ** attempt) + random.random())
            time.sleep(sleep_s)
            if attempt == max_retries - 1:
                raise

# -----------------------
# 4) Run end-to-end
# -----------------------
def main():
    df = pd.read_excel(INPUT_XLSX)
    qcol = df.columns[0]
    sample = df[qcol].dropna().sample(n=N_SAMPLE, random_state=RANDOM_SEED).astype(str).tolist()

    outputs: List[Dict[str, Any]] = []
    with open(OUTPUT_JSONL, "w", encoding="utf-8") as f:
        for i, q in enumerate(sample, start=1):
            out = call_llm(q)
            outputs.append({"user_query": q, **out})
            f.write(json.dumps({"user_query": q, **out}, ensure_ascii=False) + "\n")
            if i % 10 == 0:
                print(f"Processed {i}/{N_SAMPLE}")

    # Flatten to CSV
    flat = pd.json_normalize(outputs)
    flat.to_csv(OUTPUT_CSV, index=False)
    print("Wrote:", OUTPUT_JSONL, OUTPUT_CSV)

if __name__ == "__main__":
    main()
